<a href="https://colab.research.google.com/github/yonhwe-yang/Diplomatic-Statement-Scraper/blob/main/plugins/hanlp_demo/hanlp_demo/zh/pos_mtl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 安装

无论是Windows、Linux还是macOS，HanLP的安装只需一句话搞定：

In [3]:
!pip install hanlp -U

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 38.1 MB/s eta 0:00:00
  Created wheel for hanlp-common: filename=hanlp_common-0.0.23-py3-none-any.whl size=30819 sha256=e2c9a7fdee3a1aadba2391ffe0093254bc0263c9ab339d40d432a7ae0fb98491
  Stored in directory: /root/.cache/pip/wheels/b8/22/2d/75d505250ca2b83d52998e97db4d3ed89315138752d2e22794
  Created wheel for hanlp-trie: filename=hanlp_trie-0.0.5-py3-none-any.whl size=6816 sha256=a359c88e763b357fd896c69a18ca86728ba56e8caf42cbebf5083fb81cbf554f
  Stored in directory: /root/.cache/pip/wheels/26/a2/e2/b2a458cadfb9ff912848b0e6205bfa5ac628daa5156ccac651
  Created wheel for hanlp-downloader: filename=hanlp_downloader-0.0.25-py3-none-any.whl size=13746 sha256=643255900c41663438f955b3a7ae164d8da

## 加载模型
HanLP的工作流程是先加载模型，模型的标示符存储在`hanlp.pretrained`这个包中，按照NLP任务归类。

In [4]:
import hanlp
hanlp.pretrained.mtl.ALL # MTL多任务，具体任务见模型名称，语种见名称最后一个字段或相应语料库

{'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_small_20201223_035557.zip',
 'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_base_20201223_201906.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20210111_124159.zip',
 'CLOSE_TOK_POS_NER_SRL_UDEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20220626_175100.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_base_20210111_124519.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ERNIE_GRAM_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_ernie_gram_base_aug_20210904_145403.zip',
 'KYOTO_EVAHAN_TOK_LEM_POS_UDEP_LZH': 'https

调用`hanlp.load`进行加载，模型会自动下载到本地缓存。自然语言处理分为许多任务，分词只是最初级的一个。与其每个任务单独创建一个模型，不如利用HanLP的联合模型一次性完成多个任务：

In [5]:
import pandas as pd
import re
from tqdm import tqdm

In [6]:
HanLP = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH)


100% 467.9 MiB   1.8 MiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_base_20210111_124519.zip to /root/.hanlp/mtl
100%  41.2 KiB  41.2 KiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/transformers/electra_zh_base_20210706_125233.zip to /root/.hanlp/transformers
100%  19.4 KiB  19.4 KiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/thirdparty/file.hankcs.com/corpus/char_table.json.zip to /root/.hanlp/thirdparty/file.hankcs.com/corpus


## 词性标注
任务越少，速度越快。如指定仅执行词性标注，默认CTB标准：

In [7]:
a=HanLP(['她们每一个人都使我<U>喜悦</U>、欢笑、幸福、眉飞色舞。', '我的希望是希望张晚霞的背影被晚霞映红。'], tasks=['pos'])
type(a)

hanlp_common.document.Document

In [24]:
def preprocess(text):
    """
    从文本中提取被 <U>...</U> 标签包裹的目标词，并移除所有 <B>...</B> 标签及其内容。
    使用占位符法来精确计算目标词在最终文本中的位置。
    """
    if pd.isna(text) or not isinstance(text, str):
        return "", (0, 0), ""

    # 1. 移除 <B>...</B> 及其内部的所有内容
    text_no_b = re.sub(r"<B>.*?</B>", "", text)

    # 2. 查找 <U>...</U> 并用一个唯一的占位符替换，同时记录目标词
    placeholder = "__TARGET_WORD__"
    u_match = re.search(r"<U>(.*?)</U>", text_no_b)

    if u_match:
        target = u_match.group(1)  # 提取目标词
        # 将 <U>...</U> 替换为占位符
        temp_text = re.sub(r"<U>.*?</U>", placeholder, text_no_b)

        # 3. 计算占位符在 temp_text 中的位置
        start = temp_text.find(placeholder)
        if start != -1:
            span = (start, start + len(target))
            # 4. 将占位符替换回目标词，得到最终纯文本
            plain_text = temp_text.replace(placeholder, target)
            return target, span, plain_text
        else:
            # 理论上不会发生
            return "", (0, 0), text_no_b
    else:
        # 没找到 <U>...</U>
        return "", (0, 0), text_no_b

In [25]:
def is_target_noun(text):
    """
    判断文本中被<U>...</U>标记的目标词是否为名词。

    Args:
        text (str): 包含标签的原始文本。

    Returns:
        bool: 如果目标词是名词则返回 True，否则返回 False。
    """
    target, span, plain_text = preprocess(text)
    if not target or not plain_text.strip():
        return False

    try:
        # 调用 HanLP 进行词性标注
        doc = HanLP(plain_text, tasks='pos')

        # 使用 'tok/fine' 和 'pos/ctb'
        tokens = doc['tok/fine']
        pos_tags = doc['pos/ctb']

    except Exception as e:
        print(f"❌ 调用 HanLP 失败: {e}")
        return False

    # 定位目标词并判断是否为名词
    current_position = 0
    target_found = False

    for token, pos in zip(tokens, pos_tags):
        token_length = len(token)
        token_start = current_position
        token_end = current_position + token_length

        # 检查目标词的起始位置是否落在当前 token 的范围内
        if token_start <= span[0] < token_end:
            target_found = True
            # 如果词性以 'N' 开头，则认为是名词
            if pos.startswith("N"):
                return True
            break

        current_position = token_end

    # 如果未找到目标词或其词性不是名词
    return False


In [26]:
def filter_excel_nouns(input_file, output_file, column_name="内容"):
    """
    读取Excel文件，筛选出目标词为名词的行，并保存到新文件。

    Args:
        input_file (str): 输入的Excel文件路径。
        output_file (str): 输出的Excel文件路径。
        column_name (str): 包含文本内容的列名，默认为"内容"。
    """
    # 1. 读取Excel文件
    try:
        df = pd.read_excel(input_file)
    except FileNotFoundError:
        print(f"❌ 找不到文件: {input_file}")
        return
    except Exception as e:
        print(f"❌ 读取Excel文件时出错: {e}")
        return

    if column_name not in df.columns:
        print(f"❌ 列名 '{column_name}' 不存在于文件中。可用的列有: {list(df.columns)}")
        return

    # 2. 创建一个新列来存储判断结果
    print("正在处理数据，请稍候...")
    df['is_noun'] = df[column_name].apply(is_target_noun)

    # 3. 筛选出目标词是名词的行
    noun_df = df[df['is_noun']].copy()

    # 4. 删除临时的 is_noun 列
    noun_df.drop(columns=['is_noun'], inplace=True)

    # 5. 保存到新的Excel文件
    try:
        noun_df.to_excel(output_file, index=False)
        print(f"✅ 处理完成！已将目标词为名词的行保存到: {output_file}")
        print(f"📊 原始数据共 {len(df)} 行，筛选后剩余 {len(noun_df)} 行。")
    except Exception as e:
        print(f"❌ 保存文件时出错: {e}")

In [38]:
# === 使用示例 ===
if __name__ == "__main__":
    # 设置你的文件路径
    input_excel = "/content/悲伤 文学.xlsx"  # 修改为你的输入文件路径
    output_excel = "/content/悲伤文学_名词筛选结果.xlsx"  # 修改为你的输出文件路径
    column = "内容"  # 你的Excel中存放文本的列名

    # 执行筛选
    filter_excel_nouns(input_excel, output_excel, column)

正在处理数据，请稍候...
✅ 处理完成！已将目标词为名词的行保存到: /content/悲伤文学_名词筛选结果.xlsx
📊 原始数据共 3858 行，筛选后剩余 1621 行。


In [37]:
def preprocess(text):
    """
    从文本中提取被 <U>...</U> 标签包裹的目标词，并移除所有 <B>...</B> 标签及其内容。
    使用占位符法来精确计算目标词在最终文本中的位置。
    """
    # 1. 移除 <B>...</B> 及其内部的所有内容
    text_no_b = re.sub(r"<B>.*?</B>", "", text)

    # 2. 查找 <U>...</U> 并用一个唯一的占位符替换，同时记录目标词
    placeholder = "__TARGET_WORD__"
    u_match = re.search(r"<U>(.*?)</U>", text_no_b)

    if u_match:
        target = u_match.group(1)  # 提取目标词
        # 将 <U>...</U> 替换为占位符
        temp_text = re.sub(r"<U>.*?</U>", placeholder, text_no_b)

        # 3. 计算占位符在 temp_text 中的位置
        start = temp_text.find(placeholder)
        if start != -1:
            span = (start, start + len(target))
            # 4. 将占位符替换回目标词，得到最终纯文本
            plain_text = temp_text.replace(placeholder, target)
            return target, span, plain_text
        else:
            # 理论上不会发生
            return "", (0, 0), text_no_b
    else:
        # 没找到 <U>...</U>
        return "", (0, 0), text_no_b

def debug_single_sentence(text):
    """
    调试单个句子，检查被 <U>...</U> 标签包裹的目标词是否被识别为名词。
    """
    target, span, plain_text = preprocess(text)
    if not target:
        print("❌ 句子中未找到<U>包裹的目标词")
        return None

    print(f"原句: {text}")
    print(f"处理后文本: {plain_text}")
    print(f"目标词: {target}, 在处理后文本中的位置: {span}")

    try:
        # 调用 HanLP 进行词性标注
        doc = HanLP(plain_text, tasks='pos')

        # 根据你实测的结果，使用 'tok/fine' 和 'pos/ctb'
        tokens = doc['tok/fine']
        pos_tags = doc['pos/ctb']

    except Exception as e:
        print(f"❌ 调用 HanLP 失败: {e}")
        return None

    print("分词/词性:")
    print(" ".join(f"{tok}/{pos}" for tok, pos in zip(tokens, pos_tags)))

    # 定位目标词并判断是否为名词
    cursor = 0
    is_noun = False
    target_found = False

    for token, pos in zip(tokens, pos_tags):
        token_start = cursor
        token_end = cursor + len(token)

        if token_start <= span[0] < token_end:
            target_found = True
            print(f"🔍 找到目标词所在分词: '{token}', 词性: {pos}")
            if pos.startswith("N"):
                is_noun = True
            break

        cursor = token_end

    if not target_found:
        print("❌ 未能在分词结果中定位到目标词")
        return None

    print(f"目标词 '{target}' 是否名词: {'✅ 是' if is_noun else '❌ 否'}")
    return is_noun

# === 测试 ===
if __name__ == "__main__":
    test_text = "<B>人民日报1989年04月26日</B>举国<U>悲伤</U>"
    result = debug_single_sentence(test_text)

原句: <B>人民日报1989年04月26日</B>举国<U>悲伤</U>
处理后文本: 举国悲伤
目标词: 悲伤, 在处理后文本中的位置: (2, 4)
分词/词性:
举国/NN 悲伤/VA
🔍 找到目标词所在分词: '悲伤', 词性: VA
目标词 '悲伤' 是否名词: ❌ 否
